In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tf-nightly-gpu-2.0-preview

In [2]:
import tensorflow as tf

import numpy as np
from tensorflow.keras.layers import Input, ZeroPadding2D, Dense, Dropout, Activation, Convolution2D, Reshape
from tensorflow.keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D, BatchNormalization

from tensorflow.keras import Model

from tensorflow.keras.layers import Layer, InputSpec
from tensorflow.keras import initializers
import tensorflow.keras.backend as K

print(tf.__version__)

2.0.0-dev20190522


In [3]:
# tf.debugging.set_log_device_placement(True)

In [4]:
import handshape_datasets as hd

DATASET_NAME = "lsa16"

data = hd.load(DATASET_NAME)

In [5]:
!pip install -q -U scikit-learn
from sklearn.model_selection import train_test_split

features = data[0]
labels = data[1]['y']

x_train, x_test, y_train, y_test = train_test_split(features,
                                                    labels,
                                                    test_size=0.33,
                                                    random_state=42)
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [7]:
labels.shape

(800,)

In [8]:
class DenseNet(Model):
    def __init__(self, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, 
                 dropout_rate=0.0, weight_decay=1e-4, classes=1000, batch_size=32, with_se_layers=True):
        super(DenseNet, self).__init__()

        self.eps = 1.1e-5
        
        self.with_se_layers = with_se_layers
                
        # compute compression factor
        compression = 1.0 - reduction
        
        self.concat_axis = 3

        # From architecture for ImageNet (Table 1 in the paper)
        nb_filter = 64
        nb_layers = [6,12,24,16] # For DenseNet-121

        self.initial_layers = []
        self.initial_layers.append(ZeroPadding2D((3, 3), name='conv1_zeropadding', 
                                   input_shape=(32, 32, 3), batch_size=batch_size))
        self.initial_layers.append(Convolution2D(nb_filter, 7, 2, name='conv1', use_bias=False))
        self.initial_layers.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name='conv1_bn'))
        self.initial_layers.append(Activation('relu', name='relu1'))
        self.initial_layers.append(ZeroPadding2D((1, 1), name='pool1_zeropadding'))
        self.initial_layers.append(MaxPooling2D((3, 3), strides=(2, 2), name='pool1'))
        
        self.dense_blocks = []
        self.transition_blocks = []
        self.se_dense_blocks = []  
        self.se_transition_blocks = []
        
        # Add dense blocks
        for block_idx in range(nb_dense_block - 1):
            stage = block_idx+2
            block, nb_filter = self.dense_block(stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
            self.dense_blocks.append(block)
        
            if (with_se_layers):
                self.se_dense_blocks.append(self.se_block(stage, 'dense', nb_filter))
                        
            # Add transition_block
            self.transition_blocks.append(self.transition_block(stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay))
            nb_filter = int(nb_filter * compression)
            
            if (with_se_layers):
                self.se_transition_blocks.append(self.se_block(stage, 'transition', nb_filter))

        final_stage = stage + 1
        block, nb_filter = self.dense_block(final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
        self.dense_blocks.append(block)

        if (with_se_layers):
            self.se_dense_blocks.append(self.se_block(final_stage, 'dense', nb_filter))
        
        self.final_layers = self.final_block(nb_filter, classes)       
    
    def final_block(self, nb_filter, classes):
        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name='conv_final_blk_bn'))
        block.append(Activation('relu', name='relu_final_blk'))
        block.append(GlobalAveragePooling2D(name='pool_final'))
        block.append(Dense(classes, name='fc6'))
        block.append(Activation('softmax', name='prob'))
        return block
    
    def conv_block(self, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
        conv_name_base = 'conv' + str(stage) + '_' + str(branch)
        relu_name_base = 'relu' + str(stage) + '_' + str(branch)

        # 1x1 Convolution (Bottleneck layer)
        inter_channel = nb_filter * 4  
        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_x1_bn'))
        block.append(Activation('relu', name=relu_name_base+'_x1'))
        block.append(Convolution2D(inter_channel, 1, 1, name=conv_name_base+'_x1', use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))

        # 3x3 Convolution
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_x2_bn'))
        block.append(Activation('relu', name=relu_name_base+'_x2'))
        block.append(ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding'))
        block.append(Convolution2D(nb_filter, 3, 1, name=conv_name_base+'_x2', use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))
        return block
                                        
    def se_block(self, stage, previous, nb_filter, ratio = 16):
        se_name = 'se' + str(stage) + '_' + previous
        block = []
        block.append(GlobalAveragePooling2D(name='global_average_pooling_2d_'+se_name))
        block.append(Dense(nb_filter // ratio, name='dense_relu_'+se_name))
        block.append(Activation('relu', name='relu_'+se_name))
        block.append(Dense(nb_filter, name='dense_sigmoid_'+se_name))
        block.append(Activation('sigmoid', name='sigmoid_'+se_name))
        return block
        

    def dense_block(self, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, 
                    grow_nb_filters=True):
        block = []
        for i in range(nb_layers):
            branch = i+1
            block.append(self.conv_block(stage, branch, growth_rate, dropout_rate, weight_decay))

            if grow_nb_filters:
                nb_filter += growth_rate

        return block, nb_filter
                                        
    def transition_block(self, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
        conv_name_base = 'conv' + str(stage) + '_blk'
        relu_name_base = 'relu' + str(stage) + '_blk'
        pool_name_base = 'pool' + str(stage) 

        block = []
        block.append(BatchNormalization(epsilon=self.eps, axis=self.concat_axis, name=conv_name_base+'_bn'))
        block.append(Activation('relu', name=relu_name_base))
        block.append(Convolution2D(int(nb_filter * compression), 1, 1, name=conv_name_base, use_bias=False))

        if dropout_rate:
            block.append(Dropout(dropout_rate))

        block.append(AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base))

        return block

    def call(self, inputs):
        x = inputs
        for layer in self.initial_layers:
            x = layer(x)
        i = 0
        for transition_block in self.transition_blocks:
            concat_feat = x
            for conv_block in self.dense_blocks[i]:                                    
                for layer in conv_block:                                    
                    x = layer(x)
                x = tf.concat([concat_feat, x], self.concat_axis)
                concat_feat = x
            if (self.with_se_layers):
                init = x
                for layer in self.se_dense_blocks[i]:
                    x = layer(x)
                x = tf.expand_dims(x,1)
                x = init * tf.expand_dims(x,1) 
            for layer in transition_block:                                    
                x = layer(x)
            if (self.with_se_layers):
                init = x
                for layer in self.se_transition_blocks[i]:
                    x = layer(x)
                x = tf.expand_dims(x,1)
                x = init * tf.expand_dims(x,1) 
            i += 1
        concat_feat = x
        for conv_block in self.dense_blocks[i]:                                    
            for layer in conv_block:                                    
                x = layer(x)
            x = tf.concat([concat_feat, x], self.concat_axis)
            concat_feat = x
        if (self.with_se_layers):
            init = x
            for layer in self.se_dense_blocks[i]:
                x = layer(x)
            x = tf.expand_dims(x,1)
            x = init * tf.expand_dims(x,1) 
        for layer in self.final_layers:
            x = layer(x)                                      
        return x

model = DenseNet(classes=16)
# model.load_weights(weights_path) 

In [9]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [11]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(tf.cast(images, tf.float32))
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


In [12]:
@tf.function
def test_step(images, labels):
    predictions = model(tf.cast(images, tf.float32))
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [13]:
EPOCHS = 800
    
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    if (epoch % 10 == 0):
        template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
        print (template.format(epoch+1,
                               train_loss.result(),
                               train_accuracy.result()*100,
                               test_loss.result(),
                               test_accuracy.result()*100))
    
    if (epoch % 100 == 0):
        # serialize weights to HDF5
        model.save_weights("dropout_se_model_epoch{}.h5".format(epoch))

Epoch 1, Loss: 2.774296283721924, Accuracy: 5.7835822105407715, Test Loss: 2.774871349334717, Test Accuracy: 4.545454502105713
Saved model to disk
Epoch 11, Loss: 2.7915639877319336, Accuracy: 7.157394886016846, Test Loss: 3.1303629875183105, Test Accuracy: 5.165289402008057
Epoch 21, Loss: 2.7821226119995117, Accuracy: 7.276119232177734, Test Loss: 2.972669839859009, Test Accuracy: 4.59956693649292
Epoch 31, Loss: 2.780043601989746, Accuracy: 7.384448528289795, Test Loss: 2.916815757751465, Test Accuracy: 4.337732315063477
Epoch 41, Loss: 2.641867160797119, Accuracy: 11.730978965759277, Test Loss: 2.7376818656921387, Test Accuracy: 9.28492259979248
Epoch 51, Loss: 2.2340028285980225, Accuracy: 24.85001564025879, Test Loss: 2.372903823852539, Test Accuracy: 21.130422592163086
Epoch 61, Loss: 1.9273940324783325, Accuracy: 34.93699264526367, Test Loss: 2.110305070877075, Test Accuracy: 30.14778709411621
Epoch 71, Loss: 1.6770542860031128, Accuracy: 43.33876419067383, Test Loss: 1.9135130

Epoch 631, Loss: 0.19333097338676453, Accuracy: 93.45329284667969, Test Loss: 1.642032265663147, Test Accuracy: 82.44609069824219
Epoch 641, Loss: 0.1903148889541626, Accuracy: 93.55542755126953, Test Loss: 1.6460139751434326, Test Accuracy: 82.52494049072266
Epoch 651, Loss: 0.187391459941864, Accuracy: 93.65442657470703, Test Loss: 1.6501444578170776, Test Accuracy: 82.60135650634766
Epoch 661, Loss: 0.1845564991235733, Accuracy: 93.75041961669922, Test Loss: 1.6545281410217285, Test Accuracy: 82.67546844482422
Epoch 671, Loss: 0.1818060278892517, Accuracy: 93.84355926513672, Test Loss: 1.6592994928359985, Test Accuracy: 82.74736785888672
Epoch 681, Loss: 0.17913633584976196, Accuracy: 93.9339599609375, Test Loss: 1.6646921634674072, Test Accuracy: 82.8171615600586
Epoch 691, Loss: 0.17658229172229767, Accuracy: 94.021484375, Test Loss: 1.7164568901062012, Test Accuracy: 82.80379486083984
Epoch 701, Loss: 0.18612699210643768, Accuracy: 93.80362701416016, Test Loss: 1.7022984027862549